In [1]:
!pip install transformers sentence-transformers huggingface-hub langchain_community langchain-text-splitters pypdf chromadb tiktoken sentencepiece gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.2 MB/s eta 0:00:0

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("2025WEC_sportRegulation.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

text_lines = [chunk.page_content for chunk in chunks]

print(text_lines)

['1\nPublié le/published on: 18.10.2024, application au/application from: 01.01.2025\nRÈGLEMENT SPORTIF \nDU CHAMPIONNAT DU MONDE D’ENDURANCE DE LA FIA \nSPORTING REGULATIONS \nOF THE FIA WORLD ENDURANCE CHAMPIONSHIP \n1. DISPOSITIONS GÉNÉRALES\n1.1 Préambule\n Le Championnat du Monde d’Endurance de la FIA (désigné ci-\naprès « le Championnat »), réservé aux voitures prototypes \n(Hypercar) et GT (LMGT3), résulte d’une coopération entre la FIA \net l’ACO.\n Il comprend principalement un titre de Champion du Monde \nd’Endurance Hypercar et un titre de Champion du Monde \nd’Endurance des Pilotes Hypercar. Il est constitué des Compétitions \ninscrites au calendrier dudit Championnat et pour lesquelles \nles ASN et les organisateurs (voir l’Article 2.1) ont signé une \nConvention d’Organisation telle qu’énoncée à l’Article 2.2 du \nprésent Règlement (voir l’Annexe 1 – partie D).\n Le Championnat est régi par le Code Sportif International de la \nFIA (désigné ci-après « le Code ») et ses an

In [3]:
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer

client = PersistentClient(path="./ragdb")
collection_name = "WEC_2025"
try:
    collection = client.get_collection(collection_name)
except:
    print("Collection not found, creating a new one.")
    collection = client.create_collection(collection_name)

embedingModel = SentenceTransformer("BAAI/bge-small-en-v1.5")

def text_embedding(text):
    return embedingModel.encode(text).tolist()



Collection not found, creating a new one.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
embeddings = [text_embedding(text) for text in text_lines]
ids = [str(i) for i in range(len(text_lines))]

collection.add(
    ids=ids,
    documents=text_lines,
    embeddings=embeddings,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import gradio as gr

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def answer_question(question):
    query_embedding = text_embedding(question)

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5,
    )

    context = "\n".join(results["documents"][0])  # top 5 chunks

    prompt = f"""
    Use the information enclosed in the tags <context> in order to answer to the questions encased in the <question> tags
    <context>
    {context}
    </context>
    <question>
    {question}
    </question>
    """

    answer = qa_pipeline(prompt, max_new_tokens=256)[0]['generated_text']
    return answer

def chatbot_response(message, history):
    answer = answer_question(message)
    return answer

custom_css = """
body {
    background-size: 100px 100px;
    font-family: 'Orbitron', sans-serif;
    margin: 0;
    padding: 0;
}

.gr-block.gr-box {
    background-color: rgba(0, 25, 51, 0.85) !important;
    border: 2px solid #FFCC00;
    border-radius: 12px;
    color: white;
}

.gr-textbox textarea, .gr-textbox input {
    background-color: #002147 !important;
    color: white !important;
    border: 1px solid #FFCC00 !important;
    border-radius: 6px;
}

button.gr-button {
    background-color: #FFCC00 !important;
    color: #002147 !important;
    font-weight: bold;
    border-radius: 6px;
    border: none;
    transition: 0.3s;
}
button.gr-button:hover {
    background-color: #FFD700 !important;
    color: black !important;
}

/* Chatbot bubbles */
.message.user {
    background-color: #003366 !important;
    color: white;
}
.message.bot {
    background-color: #FFCC00 !important;
    color: #002147;
    font-weight: bold;
}

/* Logo image tweaks */
.gr-image .image-fullscreen {
    display: none !important;
}
#logo img {
    box-shadow: none !important;
    border: none !important;
    border-radius: 0 !important;
}

/* Title and description */
h1, h2, .gr-markdown {
    color: #FFCC00 !important;
    text-align: center;
    font-weight: bold;
}
"""

with gr.Blocks(theme=gr.themes.Glass(), css=custom_css) as app:

    # Logo Row (centered above chatbot)
    with gr.Row(variant="compact"):
        with gr.Column():
            with gr.Row(elem_id="logo", equal_height=True):
                gr.Image("https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/WEC_Logo.svg/langfr-330px-WEC_Logo.svg.png", show_label=False, container=False,show_download_button=False,show_fullscreen_button=False,scale=0.5)
                gr.Image("https://cdn.freebiesupply.com/logos/large/2x/fia-logo-black-and-white.png", show_label=False, container=False,show_download_button=False,show_fullscreen_button=False,scale=0.5)
                gr.Image("https://laplace.ouest-france.fr/wp-content/uploads/aco-automobile-club-de-louest-logo.png", show_label=False, container=False,show_download_button=False,show_fullscreen_button=False,scale=0.5)

    # Chatbot UI
    gr.ChatInterface(
        fn=chatbot_response,
        chatbot=gr.Chatbot(label="Assistant WEC"),
        textbox=gr.Textbox(placeholder="Posez votre question ici...", label="Votre question"),
        title="Assistant Règlement WEC",
        description="🔍 Un assistant alimenté par des documents pour mieux comprendre le règlement d'endurance.",
    )

app.launch()